In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import keras
from math import sqrt
from matplotlib import pyplot
import numpy as np
import pandas as pd
import tensorflow as tf

Using TensorFlow backend.


In [2]:
def fit_lstm(Dx,Dy, nb_epoch, neurons,load_dir=None,load_old_model=False,lr=0.000001,start_epoch=0):
    if (load_old_model):
        print("loading old model.")
        model=keras.models.load_model(load_dir)
    else:
        model = Sequential()
        model.add(LSTM(neurons, batch_input_shape=(1, 1,Dx.shape[2]), stateful=True))
        model.add(Dense(units=16,activation = 'relu'))
        model.add(Dense(units=1,activation = 'relu'))    
    
    
    opt = keras.optimizers.Adam(lr = lr)
    model.compile(loss='mean_squared_error', optimizer=opt,metrics=['accuracy'])
    
    X,y = data_shuffle(Dx,Dy)
    sample_weight = y*len(y)/y.sum()+1
    for i in range(start_epoch,start_epoch+nb_epoch):
        print("epoch"+str(i))
        model.fit(X, y, epochs=1, batch_size=1, verbose=1, shuffle=False,sample_weight=sample_weight)
        model.reset_states()
        saveStr = './models/25hz/Keras_model/keras_LSTM_combinedData_'+str(neurons)+'_i'+str(i)+'.hdf5'
        model.save(saveStr)
        if (i%10==0):
            X,y = data_shuffle(Dx,Dy)
            predict = model.predict(X,batch_size=1,verbose=1)
            sample_weight = y*len(y)/y.sum()+1 + y*len(predict)/predict.sum()
        
    return model

In [4]:
Dx1 = np.loadtxt('./PushData/PSDSData (17_20)_raw.csv',delimiter=',')
Dy1 = np.loadtxt('./PushData/push_(17_20).csv',delimiter=',')
Dx2 = np.loadtxt('./PushData/PSDSData (30)_raw.csv',delimiter=',')
Dy2 = np.loadtxt('./PushData/push_30.csv',delimiter=',')
DataNonAct = np.loadtxt('./PushData/NonActivityData.csv',delimiter=',')
DataNonActY = np.zeros_like(DataNonAct[:,1])

In [5]:
Datax = np.empty([0,9])
Datax = np.append(Datax,Dx1,axis=0)
Datax = np.append(Datax,Dx2,axis=0)
Datax = np.append(Datax,DataNonAct,axis=0)

Datay = np.empty([0])
Datay = np.append(Datay,Dy1,axis=0)
Datay = np.append(Datay,Dy2,axis=0)
Datay = np.append(Datay,DataNonActY,axis=0)

np.savetxt('./PushData/CombineDataX.csv',Datax,delimiter=',')
np.savetxt('./PushData/CombineDatay.csv',Datay,delimiter=',')

In [6]:
Datax=np.loadtxt('./PushData/combineDatax.csv',delimiter=',')
Datay=np.loadtxt('./PushData/combineDatay.csv',delimiter=',')

In [7]:
Datay.shape

(1788937,)

In [8]:
Datax.shape

(1788937, 9)

In [10]:
Datax = Datax[:,3:9].reshape(Datax.shape[0],1,6)
Datax[:,:,3:6] = np.arccos(Datax[:,:,3:6]/10.0)

In [11]:
def data_shuffle(x,y,gap=150,nonActGap=500):
    
    label_index= np.where(y==1)[0]
    label_index = np.append(label_index,[len(y)-1])
    segments=np.zeros([0,2],dtype=int)
    last_seg = 0

    for i in range(len(label_index) -1):
        if ((label_index[i+1]-label_index[i])>gap):
            num_seg = int((label_index[i+1]-label_index[i])/nonActGap)+2
            seg_gap = int((label_index[i+1]-label_index[i])/num_seg)
            for j in range(num_seg):
                if j == num_seg-1:  # last segment
                    seg = label_index[i+1]
                else:
                    seg = seg_gap*(j+1)+label_index[i]
                segments = np.append(segments,[[last_seg,seg]],axis=0)
                last_seg=seg
    
    newSeg = np.arange(len(segments))
    np.random.shuffle(newSeg)
    newx = np.zeros([0,x.shape[1],x.shape[2]])
    newy = np.zeros([0])
    #print("number of segments:"+str(len(newSeg)))
    for i in range(len(newSeg)):
        newx = np.append(newx,x[segments[newSeg[i]][0]:segments[newSeg[i]][1]],axis=0)
        newy = np.append(newy,y[segments[newSeg[i]][0]:segments[newSeg[i]][1]],axis=0)
    return newx,newy

In [ ]:
start_epoch=100
model_dir = './models/25hz/Keras_model/keras_LSTM'+str(64)+'_i'+str(start_epoch)+'.hdf5'

lstm_model=fit_lstm(Datax,Datay,500,64,load_dir=model_dir,load_old_model=True,lr=0.000001,start_epoch=start_epoch)

loading old model.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
epoch100
Epoch 1/1
1098092/1788936 [=================>............] - ETA: 3:39